# Arp peculiars galaxies catalog __ First steps with Multi-Order Coverage map's

Stefania Amodeo¹, Katarina A. Lutz¹, Manon Marchand¹

1. Université de Strasbourg, CNRS, Observatoire Astronomique de Strasbourg, UMR 7550, F-67000, Strasbourg, France

***

## Introduction

In this tutorial, we will explore Arp's Catalog of peculiar galaxies. Using Multi-Order Coverage maps, we will find in seconds wich galaxies are present in the Sloan Digital Sky Survey ([SDSS](https://www.sdss.org/ "https://www.sdss.org/")) and have been observed by the Galaxy Evolution Explorer ([GALEX](https://galex.stsci.edu/GR6/ "https://galex.stsci.edu/GR6/")) satellite.

![<img src="Data/images/joachim_dietrich_02.jpg" width="20"/>](Data/images/joachim_dietrich_02.jpg)

Figure: Arp147 taken by Joachim Dietrich. Credit: NASA & ESA

In [1]:
# Astronomy tools
import astropy.units as u
from astropy.coordinates import SkyCoord

# Access astronomical databases
import pyvo
from astroquery.cds import cds
from astroquery.vizier import Vizier

# Sky visualization
from ipyaladin import Aladin

## Find and download tables from VizieR

[![Vizier](https://custom-icon-badges.demolab.com/badge/Vizier-gray.svg?logo=vizier&logoColor=orange&logoWidth=20)](https://vizier.cds.unistra.fr/viz-bin/VizieR "https://vizier.cds.unistra.fr/viz-bin/VizieR")

We use the `Vizier` sub-module from the `astroquery` module. 

We will explore the [Arp's Catalog of Peculiar Galaxies](http://arpgalaxy.com/) compiled by [Halton C. Arp](https://www.haltonarp.com/) and published by Denis Webb.

Hence, we ask `Vizier` to find all catalogues that have a match with the keywords 'Arp Galaxies' and write the result into the variable `catalog_list_arp` (first line of code). Then we tell Python to `print` out the query result in a readable way (second line of code).  

In [2]:
catalog_list_arp = Vizier.find_catalogs("peculiar galaxies")
for name, item in catalog_list_arp.items():
    print(name, ": ", item.description)

VI/76 :  Simulation Atlas of Tidal Features in Galaxies (Howard+ 1993)
VII/74A :  Atlas of Peculiar Galaxies (Arp 1966)
VII/170 :  Catalogue of Southern Peculiar Galaxies and Associations (Arp+, 1987)
VII/192 :  Arp's Peculiar Galaxies (Webb 1996)
VII/198 :  Mark III Catalog of Galaxy Peculiar Velocities (Willick+ 1997)
J/ApJ/630/784 :  UBVR observations of nearby galaxies (Taylor+, 2005)
J/ApJ/645/228 :  Velocities of satellites of external disk galaxies (Azzaro+ 2006)
J/ApJ/663/734 :  Interacting Galaxies in GEMS and GOODS (Elmegreen+, 2007)
J/ApJ/769/39 :  SN Ibn PS1-12sk optical and NIR light curves (Sanders+, 2013)
J/ApJ/872/202 :  CFHT ugi photometry of globular clusters in M85 (Ko+, 2019)
J/ApJ/897/183 :  Chemical abundances of 3 stars in Grus II galaxy (Hansen+, 2020)
J/ApJ/903/110 :  Radial velocity and g-i color in M85 globular clusters (Ko+, 2020)
J/ApJ/921/91 :  Catastrophic cooling in superwinds. II. (Danehkar+, 2021)
J/ApJS/81/413 :  Southern sky survey of 1355 spiral gal

The catalogue that we are interested in today is `VII/192` and its description is "Arp's Peculiar Galaxies (Webb 1996)". In the TAP query we already see that there are two tables in this catalogue: `VII/192/arplist` and `VII/192/arpord`. To be able to work with these data, let's load them into this notebook. 

Again we can use either `astroquery` or `pyvo`. Which one you chose is up to you and which one you like better.

In oder to get the full catalogue with `astroquery`, we first set the row limit to infinite (*i.e.* `-1` in the notation of this package) and then ask `Vizier` to write the content of the catalogue into the variable `catalogs_arp`:

In [3]:
catalogs_arp = Vizier(row_limit=-1).get_catalogs("VII/192")
catalogs_arp

TableList with 2 tables:
	'0:VII/192/arpord' with 12 column(s) and 338 row(s) 
	'1:VII/192/arplist' with 13 column(s) and 592 row(s) 

Now let's inspect, what we got:

In [4]:
for catalog in catalogs_arp:
    print(f"{catalog.meta['ID']}: {catalog.meta['description']}")

VII_192_arpord: list of Arp views with imaging data
VII_192_arplist: list and info for involved galaxies


As we have seen in the first part of the tutorial, the catalogue "Arp's Peculiar Galaxies (Webb 1996)" comes with two tables: arpord and arplist. As you can see from the printout `Vizier` has downloaded both of them. However, we are still only interested in the arplist (index `1` in the TableList). Therefore, we write only the arplist table into a new variable `table_arplist` (first line) and then display the table (second line):

In [5]:
table_arplist = catalogs_arp[1]
table_arplist

Arp,Name,VT,u_VT,dim1,dim2,u_dim2,MType,Uchart,RAJ2000,DEJ2000,Simbad,NED
,,mag,,arcmin,arcmin,,,,,,,
int16,str16,float32,str1,float32,float32,str1,str14,int16,str10,str9,str6,str3
249,UGC 12891,16.2,,--,--,,,--,00 00 19.3,+22 59 26,Simbad,NED
249,UGC 12891,16.2,,1.3,0.5,,,--,00 00 21.6,+22 59 42,Simbad,NED
112,NGC 7805,13.3,,1.2,0.9,,SAB0^0: pec,89,00 01 26.9,+31 26 02,Simbad,NED
112,NGC 7806,13.5,,1.1,0.8,,SA(rs)bc? pec,89,00 01 30.2,+31 26 33,Simbad,NED
130,IC 5378,15.6,,0.5,--,,SBc,--,00 02 37.7,+16 39 08,Simbad,NED
130,IC 5378,15.3,,--,--,,E,--,00 02 37.8,+16 38 37,Simbad,NED
51,MGC-02-01-24,15.0,,0.8,--,,,--,00 06 16.8,-13 26 53,Simbad,NED
144,NGC 7828,14.4,,0.9,0.5,,Ring A,260,00 06 27.1,-13 24 58,Simbad,NED


Nice, we got the data we want. Before moving on, let's have a look at how to use `pyVO` to get the same table wiht a Table Access Protocol ([TAP](https://ivoa.net/documents/TAP/20190927/index.html "https://ivoa.net/documents/TAP/20190927/index.html")) query. 

First, we set the `TAPService` of `pyvo` to the server of Vizier that will receive our TAP query.

Then we write a simple query telling VizieR that we want all the rows and all (all is written `*` in SQL) the columns from the table `VII/192/arplist`. Note that we have to encompass the table name with quotation marks due to the special character `/` in the table name. 

In [6]:
tap_vizier = pyvo.dal.TAPService("https://tapvizier.cds.unistra.fr/TAPVizieR/tap/")
query = """SELECT  *  FROM "VII/192/arplist" """
table_arplist = tap_vizier.search(query).to_table()
table_arplist

recno,Arp,Name,VT,u_VT,dim1,dim2,u_dim2,MType,Uchart,RAJ2000,DEJ2000
,,,mag,,arcmin,arcmin,,,,deg,deg
int32,int16,object,float64,str1,float64,float64,str1,object,int16,float64,float64
112,333,NGC 1024,12.1,,3.9,1.4,,(R')SA(r)ab,175,39.80083333333333,10.847222222222221
120,200,NGC 1134,12.1,,2.5,0.9,,S?,175,43.42124999999999,13.015277777777776
119,190,UGC 02320,15.2,,0.5,0.3,,Multiple Sys,--,42.58291666666666,12.889444444444443
89,290,IC 0195,14.3,,1.6,0.8,,S0,--,30.935833333333328,14.708611111111109
90,290,IC 0196,14.2,,2.8,1.4,,S0-,--,30.95833333333333,14.73972222222222
111,258,UGC 02140A,15.5,,0.9,0.2,,SB.0*/,--,39.78874999999999,18.367499999999996
110,258,UGC 02140,15.4,,1.7,0.7,,IBS9P,--,39.77583333333333,18.382777777777775
109,258,Hickson 18C,16.1,,--,--,,S?,--,39.774583333333325,18.388333333333332


There we go, we now know two nifty ways to get VizieR tables with Python. Obviously there are many other things you can do with this tools. For example if you set your TAP endpoint to `https://simbad.cds.unistra.fr/simbad/sim-tap` instead of the VizieR enpoint, you can query SIMBAD just like that. 

[![Aladin](https://custom-icon-badges.demolab.com/badge/Aladin-gray.svg?logo=aladin&logoColor=purple&logoWidth=20)](https://aladin.cds.unistra.fr/aladin.gml "https://aladin.cds.unistra.fr/aladin.gml")

Note that not all rows but only the first and last 10 rows for all columns are shown in the display. To finish off this little excursion, we now want to visualise the location of the entries of this table in an Aladin Lite widget. To do so, we tell Python to take the variable `aladin` and add the table `table_arplist` to it:

In [7]:
aladin = Aladin(show_fullscreen_control=True, height=600)
aladin

Aladin(height=600.0, options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_survey'…

In [8]:
aladin.add_table(table_arplist)

Now explore the Aladin Lite widget. You will find that the location of the sources in the arplist table are marked with coloured symbols. You can zoom in and out to look at the different sources and check their peculiarity. If you click on one of the colour symbols, you will be able to see the corresponding row in the bottom of the Aladin Lite widget. 

Note if you are using Jupyter Lab instead of single Jupyter notebooks, you can open two notebooks that share a kernel. Then one notebook could contain your working environment, where you get and work on your tables ect. The other notebook could contain the Aladin Lite widget, and just remain visible all the time. This way there would be no need to scroll up and down. 

## MOCs in Python

We can use MOCs to find out whether any of these galaxies have been observed both by SDSS and GALEX. For this task we use the `cds` module of `astroquery` and the functionalities of `MOCpy` to get the intersection of the MOCs. First let's query the MOC server for all things SDSS and all things GALEX.

In [9]:
info_sdss = cds.find_datasets(meta_data="ID=*SDSS*")
info_sdss[["ID"]]

ID
str25
CDS/J/ApJ/749/10/SDSS-obs
CDS/P/HLA/SDSSg
CDS/P/HLA/SDSSr
CDS/P/HLA/SDSSz
CDS/P/HST/SDSSg
CDS/P/HST/SDSSr
CDS/P/HST/SDSSz
CDS/P/SDSS9/color
CDS/P/SDSS9/color-alt


In [10]:
info_galex = cds.find_datasets(meta_data="ID=*GALEX*")
info_galex[["ID"]]

ID
str24
CDS/P/GALEXGR6/AIS/FUV
CDS/P/GALEXGR6/AIS/NUV
CDS/P/GALEXGR6/AIS/color
CDS/P/GALEXGR6_7/FUV
CDS/P/GALEXGR6_7/NUV
CDS/P/GALEXGR6_7/color


Now that we know what the relevant data sets are called, we can move on to download the corresponding MOCs,

In [11]:
moc_sdss = cds.find_datasets(meta_data="ID=CDS/P/SDSS9/color", return_moc=True)
moc_galex = cds.find_datasets(meta_data="ID=CDS/P/GALEXGR6/AIS/color", return_moc=True)

and calculate the intersection of these two MOCs.

In [12]:
moc_intersection = moc_sdss & moc_galex
print(
    f"The intersection of SDSS and GALEX covers"
    f" {round(moc_intersection.sky_fraction *100., 1)}% of the sky",
)

The intersection of SDSS and GALEX covers 31.5% of the sky


Now we can use this MOC to filter our table of Peculiar Galaxies. 

In [13]:
coords = SkyCoord(
    ra=table_arplist["RAJ2000"],
    dec=table_arplist["DEJ2000"],
    unit=(u.deg, u.deg),
)
mask = moc_intersection.contains_skycoords(coords)
print(
    f"Among the {len(table_arplist)} peculiar galaxies, {len(table_arplist[mask])} are observed by both GALEX and SDSS!",
)

Among the 592 peculiar galaxies, 425 are observed by both GALEX and SDSS!


`mask` is a boolean table that is `True` where a peculiar galaxy is contained in the intersection of SDSS and GALEX MOCs. Doing `table_arplist[mask]` results in a smaller table corresponding to the lines that are `True` in `mask`.

To visualise only the galaxies within the MOC, we can add the filtered table to the AladinLite widget. This table will show up in a different colour than the first one we visualised.

In [14]:
aladin.add_table(table_arplist[mask])

The last thing to do with our MOC is to visualise it. We can either plot it using `matplotlib` (see the MOCpy documentation, how to do this) or add it to our AladinLite widget:

In [15]:
aladin.add_moc_from_dict(
    moc_intersection.serialize(format="json"),
    {"color": "red", "opacity": 0.45},
)

Before leaving the tutorial, don't forget to scroll back up and look at the results in the Aladin widget ;)